In [1]:
# import os
# import json

# def load_or_initialize_json(filepath, default=None):
#     if default is None:
#         default = []
#     if os.path.exists(filepath):
#         with open(filepath, "r") as f:
#             return json.load(f)
#     return default

# SCRAPED_CONTENTS_FILE = "scraped_contents.json"
# scraped_contents = load_or_initialize_json(SCRAPED_CONTENTS_FILE)
# text_contents = [tweet['content'] for tweet in scraped_contents]
# print(len(text_contents))
# print(type(text_contents))

import pandas as pd

df = pd.read_pickle("data/extracted_cleaned_df.pkl")
print(df.columns)

list_semantic_columns = [
    "job_title",
    "job_skills",
    "city",
    "country",
    "job_listing_text",
    "company_info",
    "role_description",
    "preferred_skills",
    "regions_or_states",
    "location_based_eligibility",
]

Index(['job_title', 'job_skills', 'apply_url', 'city', 'country', 'company_id',
       'involves_ai', 'jobs_towardsai_url', 'job_listing_text', 'company_name',
       'company_info', 'role_description', 'preferred_skills',
       'salary_currency', 'job_type_answer', 'experience_years',
       'experience_level', 'regions_or_states', 'location_based_eligibility',
       'remote_answer', 'salary_min', 'salary_max'],
      dtype='object')


### Embed a list of strings

In [2]:
import os
import cohere
import numpy as np
from dotenv import load_dotenv

load_dotenv()

model_name = "embed-english-v3.0"
# api_key: str | None = os.getenv("COHERE_API_KEY")
input_type_embed = "search_document"

co = cohere.Client()
array_embeds_columns = np.zeros((10, 10609, 1024), dtype=np.float64)
print(array_embeds_columns.shape)

for i in range(len(list_semantic_columns)):
    text_contents = df[list_semantic_columns[i]].tolist()

    # Get the embeddings
    embeds: list[list[float]] = co.embed(
        texts=text_contents, model=model_name, input_type=input_type_embed
    ).embeddings

    # print(embeds)
    # print(type(embeds), len(embeds), len(embeds[0]), type(embeds[0]))
    # print(embeds[0])
    array_embeds = np.array(embeds)
    # print(array_embeds.shape)
    # print(array_embeds.dtype)
    array_embeds_columns[i] = array_embeds

print(array_embeds_columns.shape)

(10, 10609, 1024)
(10, 10609, 1024)


In [9]:
np.save("embeddings_10_columns.npy", array_embeds_columns)

In [10]:
import numpy as np


def cosine_similarity_matrix(vectors, query_vec):
    dot_product = np.dot(vectors, query_vec)

    norms_vectors = np.linalg.norm(vectors, axis=1)
    norm_query_vec = np.linalg.norm(query_vec)

    if norm_query_vec == 0 or np.any(norms_vectors == 0):
        raise ValueError(
            "Cosine similarity is not defined when one or both vectors are zero vectors."
        )

    similarity = dot_product / (norms_vectors * norm_query_vec)
    return similarity

### Query similar embeddings

In [15]:
import os
import cohere
from dotenv import load_dotenv

load_dotenv()

model_name = "embed-english-v3.0"
# input_type_embed = "search_query"
input_type_embed = "search_document"

QUERY = "data scientist"

co = cohere.Client()

query_embed: list[list[float]] = co.embed(
    texts=[QUERY], model=model_name, input_type=input_type_embed
).embeddings

query_array = np.array(query_embed)
query_array = query_array.reshape(-1)

embeds_dataset = np.load("embeddings_10_columns.npy")
print(embeds_dataset.shape)
embeds_one_column = embeds_dataset[0]
print(embeds_one_column.shape)

similarity_results = np.zeros((embeds_one_column.shape[0],), dtype=np.float32)
try:
    similarity_results = cosine_similarity_matrix(embeds_one_column, query_array)
except ValueError as e:
    print(e)

list_job_titles = df["job_title"].tolist()
sorted_indices = np.argsort(similarity_results)[::-1]
for i in range(1000):
    print(similarity_results[sorted_indices[i]], list_job_titles[sorted_indices[i]])

(10, 10609, 1024)
(10609, 1024)
0.999964664505225 data scientist 
0.999964664505225 data scientist
0.999964664505225 data scientist
0.999964664505225 data scientist
0.999964664505225 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist 
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 data scientist
0.9999375147874432 